In [2]:
import pandas as pd
import numpy as np 
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns', None)

# Data loading

In [3]:
data_path = Path().absolute().parent/'data'
geo_info = pd.read_csv(data_path/'geo_info.csv', sep=';')
referer_vectors = pd.read_csv(data_path/'referer_vectors.csv', sep=';')
test = pd.read_csv(data_path/'test.csv', sep=';')
test_users = pd.read_csv(data_path/'test_users.csv', sep=';')
train = pd.read_csv(data_path/'train.csv', sep=';')
train_labels = pd.read_csv(data_path/'train_labels.csv', sep=';')

# Initial analysis

In [4]:
train.info()
display(train.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   request_ts  750000 non-null  int64 
 1   user_id     750000 non-null  object
 2   referer     750000 non-null  object
 3   geo_id      750000 non-null  int64 
 4   user_agent  749999 non-null  object
dtypes: int64(2), object(3)
memory usage: 28.6+ MB


,request_ts,user_id,referer,geo_id,user_agent
461619,1700990304,b475c231c2b75b3db6c0bfee0a100702,https://62faeb4/14bba515,2521,"{'browser': 'Yandex Browser', 'browser_version..."
337150,1700981215,96393e92377712818afce68c471b2129,https://8807153/,3663,"{'browser': 'Yandex Browser', 'browser_version..."
626522,1700996568,af00842eb9463a56760d034683955345,https://8807153/,1724,"{'browser': 'Edge', 'browser_version': '119.0...."
316399,1701001472,5772d8bb50119bb7c3baf9a690627e85,https://7b8d3ae/15f1b118,9652,"{'browser': 'Chrome Mobile', 'browser_version'..."
338097,1701014783,948264a50cdd876cb7b92f41328b083b,https://8807153/,9545,"{'browser': 'Yandex Browser', 'browser_version..."


In [5]:
geo_info.info()
display(geo_info.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 0 to 5532
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   geo_id       5533 non-null   int64 
 1   country_id   5533 non-null   object
 2   region_id    3826 non-null   object
 3   timezone_id  5533 non-null   object
dtypes: int64(1), object(3)
memory usage: 173.0+ KB


,geo_id,country_id,region_id,timezone_id
1259,1015,110628b,394501,11a38d2
1198,8906,1234f1d,15a554,d903ee
881,8241,1016727,NaN,f91be1
4450,1911,c31b4e,f66ff,f6155e
37,6468,e59127,NaN,be51be


In [6]:
referer_vectors.info()
display(referer_vectors.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   component0  200000 non-null  int64 
 1   component1  200000 non-null  int64 
 2   component2  200000 non-null  int64 
 3   component3  200000 non-null  int64 
 4   component4  200000 non-null  int64 
 5   component5  200000 non-null  int64 
 6   component6  200000 non-null  int64 
 7   component7  200000 non-null  int64 
 8   component8  200000 non-null  int64 
 9   component9  200000 non-null  int64 
 10  referer     200000 non-null  object
dtypes: int64(10), object(1)
memory usage: 16.8+ MB


,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,referer
121021,12942,2426,10748,-3120,10996,4233,-4642,11445,2111,21464,https://7f39dd4/175f0df9
3804,-1070,659,4670,10459,4857,18516,-5955,13705,15733,8909,https://6a81948/16e65707
168248,18360,14496,6081,4602,7298,8148,9734,6157,8953,-10072,https://b5380d6/12d1e9e5
141873,12376,-7666,11340,5740,2779,4411,-3108,5018,-1176,25671,https://a32b6fc/13281344
127832,13651,7860,5654,12902,18347,12430,-2502,-7557,-4778,6973,https://710604b/12bf39f5


In [7]:
train_labels.info()
display(train_labels.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  500000 non-null  object
 1   target   500000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.6+ MB


,user_id,target
348690,68c6d1f7db4bf72f201292c5e31c7ba5,0
478371,89e736655614aa516d6a9173c276c5c7,1
332898,fa1166d17e5da35f3173e1d9aadb002c,1
258117,4a422ac55c6b5ce9c8a9bc0c05a2cacc,1
226844,0a5ee1ac923c254e773baa89dbfb270c,0


# Data Processing

In [8]:
# Объединим таблицы
train = pd.merge(train, geo_info, on='geo_id')
train = pd.merge(train, referer_vectors, on='referer')
train = pd.merge(train, train_labels, on='user_id')
train = train.drop(['geo_id', 'referer'], axis=1)

In [9]:
# Найдем и удалим дубликаты
print('Количество дубликатов:', train.duplicated().sum())
train = train.drop_duplicates()

Количество дубликатов: 9454


In [10]:
# Найдем и удалим пропуски
print('Кол-во пропусков:\n', train.isnull().sum())
train = train.dropna(axis=0, how='any')

Кол-во пропусков:
 request_ts         0
user_id            0
user_agent         1
country_id         0
region_id      49903
timezone_id        0
component0         0
component1         0
component2         0
component3         0
component4         0
component5         0
component6         0
component7         0
component8         0
component9         0
target             0
dtype: int64


In [11]:
train[(train['country_id']=='c31b4e') & (train['timezone_id']=='e56e80')].head(3)

,request_ts,user_id,user_agent,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,target
1,1700986581,46a5f128fd569c764a92c2eaa788095e,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,44520b,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0
3,1700992803,af735816ca19115431ae3d89518c8c91,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,3c9dca,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0
5,1700956244,b74aaac38e44f90517d3e217fa118cec,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,1fbfa5,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0


In [12]:
train[(train['region_id']=='3c9dca')].head(3)

,request_ts,user_id,user_agent,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,target
3,1700992803,af735816ca19115431ae3d89518c8c91,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,3c9dca,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,0
804,1701023748,5b3cdd0eba30838fb01deb702845cdf4,"{'browser': 'Chrome Mobile', 'browser_version'...",c31b4e,3c9dca,e56e80,13294,3884,10452,-1916,11369,6138,-2472,10123,3029,21118,1
1024,1701019068,18232c8806188f0adeb265e4600223af,"{'browser': 'Chrome', 'browser_version': '118....",c31b4e,3c9dca,e56e80,16220,2266,9404,-3442,11485,4129,-3919,6506,2556,21868,0


In [13]:
train = train.drop(['country_id', 'timezone_id'], axis=1)

Признаки country_id и timezone_id не информативны, поскольку при их фиксированных значениях параметр region_id имеет различные значения. А фиксированному region_id соответствуют одни и те же единственные показатели country_id и timezone_id. Поэтому мы можем исключить эти столбцы.

In [14]:
# Раскроем словарь столбца user_agent
import ast
list = ['browser', 'browser_version', 'os', 'os_version']
for key in list:
    train[key] = train['user_agent'].apply(lambda x: ast.literal_eval(x)[key])
train = train.drop('user_agent', axis=1)

In [15]:
# Ввиду высокой смысловой корреляции между browser и os,удаляем последний, тк он, к тому же, менее информативен
train = train.drop(['os', 'browser_version', 'os_version'], axis=1)

In [16]:
train = train.drop(['user_id'], axis=1)

In [17]:
# выделим топ 25 значений столбца browser
top_25 = [x for x in train['browser'].value_counts().sort_values(ascending=False).head(25).index]
train['browser'] = train['browser'].apply(lambda x: x if x in top_25 else 'Other')

In [18]:
# преобразуем столбец дата-время request_ts
train['request_date'] = pd.to_datetime(train['request_ts'], unit='s')
train.insert(loc=1,column='request_year', value=train['request_date'].dt.year)
train.insert(loc=2, column='request_month', value=train['request_date'].dt.month)
train.insert(loc=3, column='request_dayofweek', value=(train['request_date'].dt.dayofweek)+1)
train.insert(loc=4, column='request_hour', value=train['request_date'].dt.hour)

print('Number of unique year: ', train['request_year'].nunique())
print('Number of unique month: ', train['request_month'].nunique())
print('Number of unique dayofweek: ', train['request_dayofweek'].nunique())
print('\n',train['request_dayofweek'].value_counts())
print('\nNumber of unique hour: ', train['request_hour'].nunique())

# Удалим стоблцы year, month, тк они не информативны и содержат единственное значение. Столбец request_dayofweek, поскольку он содержит только дни 6 и 7, которые коррелируют как выходные. А также исходный request_ts и request_date
train = train.drop(['request_year', 'request_month', 'request_ts', 'request_dayofweek', 'request_date'], axis=1)

Number of unique year:  1
Number of unique month:  1
Number of unique dayofweek:  2

 request_dayofweek
7    510348
6     31584
Name: count, dtype: int64

Number of unique hour:  24


In [19]:
# нормализуем столбцы
numeric_features = ['component0', 'component1', 'component2', 'component3', 'component4', 'component5', \
                    'component6', 'component7', 'component8', 'component9']
time_feature = ['request_hour']
mean_encoding_feature = ['region_id']


def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))
def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

one_hot_encoder = OneHotEncoder(drop='first', handle_unknown='ignore',sparse_output=False)

target = TargetEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', one_hot_encoder, ['browser']),
        ("hour_sin", sin_transformer(24), ["request_hour"]),
        ("hour_cos", cos_transformer(24), ["request_hour"]),
        ('trgt', target, ['region_id'])
    ])

preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['component0', 'component1', 'component2',
                                  'component3', 'component4', 'component5',
                                  'component6', 'component7', 'component8',
                                  'component9']),
                                ('cat',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['browser']),
                                ('hour_sin',
                                 FunctionTransformer(func=<function sin_transformer.<locals>.<lambda> at 0x0000014A8286AB60>),
                                 ['request_hour']),
                                ('hour_cos',
                                 FunctionTransformer(func=<function cos_transformer.<locals>.<lambda> at 0x0000014A8286AAC0>),
                                 ['request_hour']),
                                ('trgt', TargetEncoder(), ['region_id'])])

In [20]:
# определим целевую переменную и обучающий набор
X = train.drop('target', axis=1)
y = train['target']

In [21]:
df_pandas = preprocessor.fit_transform(X, y)

# Model for predicting gender

In [89]:
# Загрузим дополнительные библиотеки
from tqdm import tqdm
import time
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from termcolor import colored

In [90]:
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df_pandas, y, test_size=0.2, random_state=42)

In [92]:
# Создадим списки для хранения метрик
train_accuracies = []
test_accuracies = []
training_times = []

# Обучим модель
start_time = time.time()
for n_estimators in tqdm(range(1, 101), desc="Обучение модели"):
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    
    # Засекаем время обучения
    estimator_start_time = time.time()
    model.fit(X_train, y_train)
    estimator_time = time.time() - estimator_start_time
    
    # Оценка точности на обучающей и тестовой выборках
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))

    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    training_times.append(estimator_time)

    # Вывод информации о прогрессе
    elapsed_time = time.time() - start_time
    print(f'Эпоха {n_estimators:<3}/100 - Время: {elapsed_time:.2f} секунд - '
          f'Точность на обучающей выборке: {train_accuracy:.4f} - '
          f'Точность на тестовой выборке: {test_accuracy:.4f}')

Обучение модели:   1%|          | 1/100 [00:02<04:43,  2.86s/it]

Эпоха 1  /100 - Время: 2.86 секунд - Точность на обучающей выборке: 0.8960 - Точность на тестовой выборке: 0.7461


Обучение модели:   2%|▏         | 2/100 [00:08<07:04,  4.33s/it]

Эпоха 2  /100 - Время: 8.22 секунд - Точность на обучающей выборке: 0.8983 - Точность на тестовой выборке: 0.7492


Обучение модели:   3%|▎         | 3/100 [00:15<09:22,  5.80s/it]

...

Эпоха 98 /100 - Время: 9740.72 секунд - Точность на обучающей выборке: 0.9800 - Точность на тестовой выборке: 0.8023


Обучение модели:  99%|█████████▉| 99/100 [2:45:33<03:07, 187.22s/it]

Эпоха 99 /100 - Время: 9933.06 секунд - Точность на обучающей выборке: 0.9800 - Точность на тестовой выборке: 0.8023


Обучение модели: 100%|██████████| 100/100 [2:48:46<00:00, 101.27s/it]

Эпоха 100/100 - Время: 10126.52 секунд - Точность на обучающей выборке: 0.9800 - Точность на тестовой выборке: 0.8023


In [94]:
# Построим графики
plt.figure(figsize=(15, 10))

# График 1: Обучающая и тестовая точность
plt.subplot(3, 1, 1)
plt.plot(range(1, 101), train_accuracies, label='Обучающая точность', color='blue')
plt.plot(range(1, 101), test_accuracies, label='Тестовая точность', color='orange')
plt.xlim(1, 100)
plt.ylim(0, 1)
plt.xlabel('Количество деревьев')
plt.ylabel('Точность')
plt.title('Точность модели в зависимости от количества деревьев')
plt.legend()
plt.grid()

# График 2: Время обучения
plt.subplot(3, 1, 2)
plt.plot(range(1, 101), training_times, label='Время обучения (сек)', color='green')
plt.xlim(1, 100)
plt.xlabel('Количество деревьев')
plt.ylabel('Время (сек)')
plt.title('Время обучения модели в зависимости от количества деревьев')
plt.legend()
plt.grid()

# График 3: Разница между обучающей и тестовой точностью
plt.subplot(3, 1, 3)
plt.plot(range(1, 101), np.array(train_accuracies) - np.array(test_accuracies), label='Разница точности', color='red')
plt.xlim(1, 100)
plt.xlabel('Количество деревьев')
plt.ylabel('Разница точности')
plt.title('Разница между обучающей и тестовой точностью')
plt.axhline(0, color='black', lw=0.5, ls='--')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

print("Обучение завершено!")

<IPython.core.display.Javascript object>

Обучение завершено!
